Import Packages

In [1]:
import copy
import pickle
import pandas as pd
from loan_default_engine_definition import BaseLoanDefaultModel

In [3]:
with open("./artefacts/input_features.pkl", "rb") as f:
    input_features = pickle.load(f)
len(input_features)

19

In [4]:
with open("./artefacts/decision_features.pkl", "rb") as f:
    decision_features = pickle.load(f)
len(decision_features)

19

In [5]:
with open("./artefacts/categorical_encoder.pkl", "rb") as f:
    categorical_encoder = pickle.load(f)
categorical_encoder

GLMMEncoder(cols=['credit_history', 'purpose', 'personal_status',
                  'other_debtors', 'property', 'installment_plan', 'housing',
                  'foreign_worker', 'job'])

In [6]:
with open("./artefacts/cal_model.pkl", "rb") as f:
    cal_model = pickle.load(f)
cal_model

CalibratedClassifierCV(cv='prefit',
                       estimator=XGBClassifier(base_score=None,
                                               booster='gbtree', callbacks=None,
                                               colsample_bylevel=0.3,
                                               colsample_bynode=0.7000000000000002,
                                               colsample_bytree=0.4,
                                               early_stopping_rounds=None,
                                               enable_categorical=False,
                                               eval_metric=None,
                                               feature_types=None, gamma=None,
                                               gpu_id=None, grow_policy=None,
                                               importance_type=None,
                                               interaction_constraints=None,
                                               learning_rate=0.4, max_bin=None,
                                               max_cat_threshold=None,
                                               max_cat_to_onehot=None,
                                               max_delta_step=None, max_depth=4,
                                               max_leaves=None,
                                               min_child_weight=6, missing=nan,
                                               monotone_constraints=None,
                                               n_estimators=176, n_jobs=None,
                                               num_parallel_tree=None,
                                               predictor=None, random_state=123, ...))

In [7]:
with open("./artefacts/shap_explainer.pkl", "rb") as f:
    shap_explainer = pickle.load(f)
shap_explainer

In [10]:
with open("./artefacts/thresholds.pkl", "rb") as f:
    thresholds = pickle.load(f)
thresholds

{'mkyc -> reject': {'threshold': 757.0,
  'Precision': 84.0,
  'Recall': 34.426,
  'group_size': 12.5},
 'approved -> mkyc': {'threshold': 103.0,
  'Precision': 1.408,
  'Recall': 1.639,
  'group_size': 35.5}}

In [11]:
mkyc_reject_threshold, approved_mkyc_threshold = thresholds.get("mkyc -> reject").get("threshold"), thresholds.get("approved -> mkyc").get("threshold")
print(mkyc_reject_threshold, approved_mkyc_threshold)

757.0 103.0


In [12]:
loan_default_engine = BaseLoanDefaultModel(
  input_feature_names = input_features,
  decision_feature_names = decision_features)
loan_default_engine.training_complete = True
loan_default_engine.MODEL_VERSION = '1.0.0'
loan_default_engine.target_encoder = copy.deepcopy(categorical_encoder)
loan_default_engine.model_obj = copy.deepcopy(cal_model.estimator)
loan_default_engine.model_calibration_obj = copy.deepcopy(cal_model)
loan_default_engine.shap_explainer_obj = copy.deepcopy(shap_explainer)
loan_default_engine.THRESHOLD_MKYC_REJECT =  mkyc_reject_threshold
loan_default_engine.THRESHOLD_APPROVED_MKYC = approved_mkyc_threshold

In [13]:
df = pd.read_csv("./data/credit.csv", index_col=0)
df.shape

(1000, 22)

In [14]:
df.head()

,checking_balance,months_loan_duration,credit_history,purpose,amount,savings_balance,employment_length,installment_rate,personal_status,other_debtors,...,age,installment_plan,housing,existing_credits,default,dependents,telephone,foreign_worker,job,gender
0,-43.0,6,critical,radio/tv,1169,NaN,13 years,4,single,none,...,67,none,own,2,0,1,2.349340e+09,yes,skilled employee,male
1,75.0,48,repaid,radio/tv,5951,89.0,2 years,2,NaN,none,...,22,none,own,1,1,1,NaN,yes,skilled employee,female
2,NaN,12,critical,education,2096,24.0,5 years,2,single,none,...,49,none,own,1,0,2,NaN,yes,unskilled resident,male
3,-32.0,42,repaid,furniture,7882,9.0,5 years,2,single,guarantor,...,45,none,for free,1,0,2,NaN,yes,skilled employee,male
4,-23.0,24,delayed,car (new),4870,43.0,3 years,3,single,none,...,53,none,for free,2,1,2,NaN,yes,skilled employee,male


In [15]:
decisions = df.apply(lambda row: loan_default_engine.decide(row), axis=1, result_type = 'expand')
decisions.head()

ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated, use `iteration_range` or model slicing instead.
ntree_limit is deprecated

,loan_default_rating,loan_default_engine_decision,loan_default_reason
0,40,approved,[]
1,664,mkyc,[]
2,19,approved,[]
3,719,mkyc,[]
4,793,reject,[CHECKING_BALANCE]


In [16]:
decisions.groupby(['loan_default_engine_decision'])[['loan_default_engine_decision']].count()/df.shape[0]

,loan_default_engine_decision
loan_default_engine_decision,
approved,0.412
mkyc,0.463
reject,0.125


In [17]:
decisions[decisions.loan_default_engine_decision=='reject']

,loan_default_rating,loan_default_engine_decision,loan_default_reason
4,793,reject,[CHECKING_BALANCE]
11,843,reject,[CHECKING_BALANCE]
15,814,reject,[RESIDENCE_HISTORY_MONTHS]
17,900,reject,[CREDIT_HISTORY]
29,855,reject,[CHECKING_BALANCE]
...,...,...,...
972,907,reject,[CREDIT_HISTORY]
973,886,reject,[CHECKING_BALANCE]
979,846,reject,[CREDIT_HISTORY]
986,878,reject,[CREDIT_HISTORY]


In [18]:
with open("./artefacts/loan_default_engine.pkl", "wb") as f:
    pickle.dump(loan_default_engine, f, pickle.HIGHEST_PROTOCOL)

In [19]:
df.columns

Index(['checking_balance', 'months_loan_duration', 'credit_history', 'purpose',
       'amount', 'savings_balance', 'employment_length', 'installment_rate',
       'personal_status', 'other_debtors', 'residence_history', 'property',
       'age', 'installment_plan', 'housing', 'existing_credits', 'default',
       'dependents', 'telephone', 'foreign_worker', 'job', 'gender'],
      dtype='object')